In [20]:
import os
import platform
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [21]:
os.getcwd()

'/Users/youngseoklee/Desktop/workplace/dacon_energy'

In [22]:
#파일경로 설정
if 'macOS' in platform.platform():
    path = '/Users/youngseoklee/Desktop/workplace/datas/dacon_energy/'
    print(path)
elif 'Linux' in platform.platform():
    path = '/workplace/datas/dacon_energy/'
    print(path)
else:
    print('어느 os에도 속해있지 않습니다')

/Users/youngseoklee/Desktop/workplace/datas/dacon_energy/


In [23]:
train_df = pd.read_csv(path + 'train_df.csv', index_col=0)
test_df = pd.read_csv(path + 'test_df.csv', index_col=0)

sub = pd.read_csv(path + 'sample_submission.csv', index_col=0)

In [24]:
train_df.shape , test_df.shape

((204000, 27), (16800, 26))

In [25]:
train_df.keys()

Index(['건물번호', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)', 'Year',
       'Month', 'Day', 'Hour', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)',
       'ESS저장용량(kWh)', 'PCS용량(kW)', '건물유형_건물기타', '건물유형_공공', '건물유형_대학교',
       '건물유형_데이터센터', '건물유형_백화점및아울렛', '건물유형_병원', '건물유형_상용', '건물유형_아파트',
       '건물유형_연구소', '건물유형_지식산업센터', '건물유형_할인마트', '건물유형_호텔및리조트'],
      dtype='object')

In [26]:
SEQUENCE_LENGTH = 24
TRAIN_SIZE = 0.9

# Select all columns and convert to numpy array
X = train_df.drop('전력소비량(kWh)', axis=1).values
y = train_df['전력소비량(kWh)'].values



# Generate sequences
#X = [X[i:i+SEQUENCE_LENGTH, :] for i in range(X.shape[0]-SEQUENCE_LENGTH)]
#y = [y[i+SEQUENCE_LENGTH] for i in range(y.shape[0]-SEQUENCE_LENGTH)]
X, y = np.array(X), np.array(y).reshape(-1, 1)


In [27]:
# Split into training and validation datasets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=TRAIN_SIZE, shuffle=False)

In [28]:
X_train.shape, y_train.shape

((183600, 26), (183600, 1))

# Modeling

In [29]:
import optuna
import lightgbm as lgb
import xgboost as xgb
from torchmetrics.regression import MeanAbsolutePercentageError
from sklearn.metrics import mean_absolute_percentage_error

In [30]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label = y_val)
dtest = xgb.DMatrix(test_df)

In [ ]:
'''
파라미터들의 정보를 알기 위해서는 어떻게 해야할까??

DMatrix로 묶고 train에 넣었을 때 왜 안되는 걸까?
'''

In [44]:
xgb.get_config()

{'use_rmm': False, 'verbosity': 2}

In [54]:
def objective(trial):
    params = {
        'booster':'dart',
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'eta': trial.suggest_loguniform('eta', 0.01, 1.0),
        'lambda':trial.suggest_loguniform('lambda', 0.1, 1.0),
        'alpha':trial.suggest_loguniform('alpha', 0.1, 1.0) ,
        'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        
    }

    bst = xgb.train(params, dtrain, evals=[(dval, 'validation')],early_stopping_rounds=100, num_boost_round=100)
    preds = bst.predict(xgb.DMatrix(X_val))
    mape = mean_absolute_percentage_error(y_val, preds)
    print("MAPE: {}".format(mape))
    return mape

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print(study.best_trial)
print(study.best_value)
# Print best parameters
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-08-09 15:23:26,760] A new study created in memory with name: no-name-7db26a48-871f-4d9f-a58e-dfa904a93c63


[15:23:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 142 extra nodes, 0 pruned nodes, max_depth=9
[15:23:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[0]	validation-rmse:1452.16493
[15:23:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 76 extra nodes, 0 pruned nodes, max_depth=9
[15:23:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[1]	validation-rmse:1355.67218
[15:23:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 14 extra nodes, 0 pruned nodes, ma

/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_45309/1956893324.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 1.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_45309/1956893324.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trial.suggest_loguniform('lambda', 0.1, 1.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_45309/1956893324.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float`

[15:23:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 188 extra nodes, 0 pruned nodes, max_depth=9
[15:23:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[6]	validation-rmse:881.49624
[15:23:27] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 382 extra nodes, 0 pruned nodes, max_depth=9
[15:23:27] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[7]	validation-rmse:829.52182
[15:23:27] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 20 extra nodes, 0 pruned nodes, max

[15:23:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 154 extra nodes, 0 pruned nodes, max_depth=9
[15:23:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[27]	validation-rmse:471.38226
[15:23:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 592 extra nodes, 0 pruned nodes, max_depth=9
[15:23:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[28]	validation-rmse:470.71493
[15:23:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, ma

...

[15:25:47] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:25:47] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[51]	validation-rmse:566.81503
[15:25:47] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:25:47] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[52]	validation-rmse:566.76424
[15:25:47] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_de

[15:25:50] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:25:50] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[72]	validation-rmse:540.08958
[15:25:50] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:25:50] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[73]	validation-rmse:488.82656
[15:25:50] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_de

[15:25:54] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:25:54] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[93]	validation-rmse:465.43321
[15:25:54] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:25:54] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/gbm/gbtree.cc:922: drop 0 trees, weight = 1
[94]	validation-rmse:448.76700
[15:25:54] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 6 extra nodes, 0 pruned nodes, max_de

[I 2023-08-09 15:25:55,866] Trial 9 finished with value: 513370333141080.75 and parameters: {'max_depth': 2, 'eta': 0.29288714558848084, 'lambda': 0.2766163014469653, 'alpha': 0.23621094389026273, 'gamma': 0.011851554941524182, 'subsample': 0.6382160884140701, 'colsample_bytree': 0.4549152810810223}. Best is trial 4 with value: 311022357494870.2.


MAPE: 513370333141080.75
FrozenTrial(number=4, state=TrialState.COMPLETE, values=[311022357494870.2], datetime_start=datetime.datetime(2023, 8, 9, 15, 24, 33, 510306), datetime_complete=datetime.datetime(2023, 8, 9, 15, 24, 48, 913322), params={'max_depth': 8, 'eta': 0.6899483235544007, 'lambda': 0.6575800041578816, 'alpha': 0.8628873055144217, 'gamma': 0.5686169027650068, 'subsample': 0.8152575285029862, 'colsample_bytree': 0.21766458645071274}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=10, log=False, low=1, step=1), 'eta': FloatDistribution(high=1.0, log=True, low=0.01, step=None), 'lambda': FloatDistribution(high=1.0, log=True, low=0.1, step=None), 'alpha': FloatDistribution(high=1.0, log=True, low=0.1, step=None), 'gamma': FloatDistribution(high=1.0, log=True, low=0.01, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.1, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.1,

In [57]:
best_params = {
    'booster':'dart',
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    trial.params
}

In [63]:
bst = xgb.train(best_params, dtrain, evals=[(dval, 'validation')],early_stopping_rounds=100, num_boost_round=1000)
y_preds = bst.predict(xgb.DMatrix(X_val))
mape = mean_absolute_percentage_error(y_val, y_preds)
print("MAPE: {}".format(mape))

[15:32:00] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 190 extra nodes, 0 pruned nodes, max_depth=8
[0]	validation-rmse:740.35685
[15:32:00] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 136 extra nodes, 0 pruned nodes, max_depth=8
[1]	validation-rmse:569.27242
[15:32:00] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 440 extra nodes, 0 pruned nodes, max_depth=8
[2]	validation-rmse:419.56588
[15:32:00] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 220 extra nodes, 0 pruned nodes, max_depth=8
[3]	validation-rmse:418.26226
[15:32:00] INFO: /Users/runner/work/xgboost/xgbo

[15:32:01] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=8
[35]	validation-rmse:229.68860
[15:32:01] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=4
[36]	validation-rmse:229.68493
[15:32:01] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 456 extra nodes, 0 pruned nodes, max_depth=8
[37]	validation-rmse:228.24840
[15:32:01] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=8
[38]	validation-rmse:228.24573
[15:32:01] INFO: /Users/runner/work/xgboost/xgbo

[69]	validation-rmse:200.67182
[15:32:02] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 354 extra nodes, 0 pruned nodes, max_depth=8
[70]	validation-rmse:200.69241
[15:32:02] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=5
[71]	validation-rmse:200.68776
[15:32:02] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 426 extra nodes, 0 pruned nodes, max_depth=8
[72]	validation-rmse:200.50392
[15:32:02] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 86 extra nodes, 0 pruned nodes, max_depth=8
[73]	validation-rmse:200.46589
[15:32:02] INFO

[15:32:03] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 306 extra nodes, 0 pruned nodes, max_depth=8
[105]	validation-rmse:198.27307
[15:32:03] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 458 extra nodes, 0 pruned nodes, max_depth=8
[106]	validation-rmse:197.96517
[15:32:03] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 376 extra nodes, 0 pruned nodes, max_depth=8
[107]	validation-rmse:197.73400
[15:32:03] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 396 extra nodes, 0 pruned nodes, max_depth=8
[108]	validation-rmse:197.01477
[15:32:03] INFO: /Users/runner/work/xgbo

[15:32:04] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 356 extra nodes, 0 pruned nodes, max_depth=8
[140]	validation-rmse:192.33524
[15:32:04] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 424 extra nodes, 0 pruned nodes, max_depth=8
[141]	validation-rmse:192.20482
[15:32:04] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 70 extra nodes, 0 pruned nodes, max_depth=8
[142]	validation-rmse:192.20730
[15:32:04] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 290 extra nodes, 0 pruned nodes, max_depth=8
[143]	validation-rmse:192.18659
[15:32:04] INFO: /Users/runner/work/xgboo

...

[15:32:24] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 416 extra nodes, 0 pruned nodes, max_depth=8
[840]	validation-rmse:136.11138
[15:32:24] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 456 extra nodes, 0 pruned nodes, max_depth=8
[841]	validation-rmse:135.93455
[15:32:24] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 260 extra nodes, 0 pruned nodes, max_depth=8
[842]	validation-rmse:135.93152
[15:32:24] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=8
[843]	validation-rmse:135.92860
[15:32:24] INFO: /Users/runner/work/xgboo

[15:32:25] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 66 extra nodes, 0 pruned nodes, max_depth=8
[875]	validation-rmse:135.16404
[15:32:25] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 274 extra nodes, 0 pruned nodes, max_depth=8
[876]	validation-rmse:135.16276
[15:32:25] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 264 extra nodes, 0 pruned nodes, max_depth=8
[877]	validation-rmse:135.16963
[15:32:25] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 448 extra nodes, 0 pruned nodes, max_depth=8
[878]	validation-rmse:134.88889
[15:32:25] INFO: /Users/runner/work/xgboo

[15:32:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 92 extra nodes, 0 pruned nodes, max_depth=8
[910]	validation-rmse:134.29407
[15:32:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 392 extra nodes, 0 pruned nodes, max_depth=8
[911]	validation-rmse:134.28781
[15:32:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=8
[912]	validation-rmse:134.28769
[15:32:26] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 350 extra nodes, 0 pruned nodes, max_depth=8
[913]	validation-rmse:134.26882
[15:32:26] INFO: /Users/runner/work/xgboos

[15:32:27] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 474 extra nodes, 0 pruned nodes, max_depth=8
[945]	validation-rmse:132.56090
[15:32:27] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 290 extra nodes, 0 pruned nodes, max_depth=8
[946]	validation-rmse:132.54409
[15:32:27] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 212 extra nodes, 0 pruned nodes, max_depth=8
[947]	validation-rmse:132.53899
[15:32:27] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 108 extra nodes, 0 pruned nodes, max_depth=8
[948]	validation-rmse:132.53933
[15:32:27] INFO: /Users/runner/work/xgbo

[15:32:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 302 extra nodes, 0 pruned nodes, max_depth=8
[980]	validation-rmse:131.13258
[15:32:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 430 extra nodes, 0 pruned nodes, max_depth=8
[981]	validation-rmse:131.04205
[15:32:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 240 extra nodes, 0 pruned nodes, max_depth=8
[982]	validation-rmse:131.04021
[15:32:28] INFO: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 280 extra nodes, 0 pruned nodes, max_depth=8
[983]	validation-rmse:130.98376
[15:32:28] INFO: /Users/runner/work/xgbo

In [65]:
pred = bst.predict(xgb.DMatrix(test_df))

array([1968.2177 , 1965.256  , 1342.6163 , ...,  427.66803,  453.76266,
        208.63881], dtype=float32)

In [66]:
sub['answer'] = pred

sub.to_csv(path + 'xgb_sub.csv')